In [57]:
#importing libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import folium
import plotly as py
import plotly.graph_objs as go
from sklearn.cluster import KMeans
import warnings
import os
warnings.filterwarnings("ignore")
py.offline.init_notebook_mode(connected = True)


### 1. Data Munging

In [35]:
customer_data = pd.read_csv('data.csv', encoding='latin-1')
customer_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [36]:
customer_data.shape

(541909, 8)

In [37]:
# Statistical Summary
customer_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [38]:
customer_data.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [39]:
# Check for missing values
def missing_values_table(customer_data):
   
   mis_val = customer_data.isnull().sum()
   mis_val_percent = 100 * customer_data.isnull().sum() / len(customer_data)

   mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1, keys=['Missing Values', '% of Total Values'])
   
   mis_val_table_ren_columns = mis_val_table[
       mis_val_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(2)  
   
   print("Your selected dataframe has {} columns.".format(customer_data.shape[1]) + '\n' +
   "There are {} columns that have missing values.".format(mis_val_table_ren_columns.shape[0]))
   
   return mis_val_table_ren_columns

In [40]:
missing_values_table(customer_data)


Your selected dataframe has 8 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
CustomerID,135080,24.93
Description,1454,0.27


A quarter (25%) of the records do not have a specific customer assigned to them. Given the available data, it is not feasible to fill in the missing values for these entries. As a result, they serve no purpose for the current task and will be removed from the dataframe. I also delete the entires of the 'Description'column with only 0.27% missing values.

In [45]:
customer_data.dropna(subset=['CustomerID'], inplace=True, axis=0)
customer_data.dropna(subset=['Description'], inplace=True, axis=0)


In [43]:
# Check for duplicates and delete them
print(f'The number of duplicates in our dataset is {customer_data.duplicated().sum()}')


The number of duplicates in our dataset is 5225


In [44]:
customer_data.drop_duplicates(inplace = True)


In [49]:
print(f'The number of missing values in our dataset is {customer_data.isnull().sum()}')
customer_data.shape

The number of missing values in our dataset is InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


(401604, 8)

By removing these entries we end up with data that is accurate and maintains its integrity.

### 2. Data Preparation


The dataframe contains 8 variables that correspond to:

* InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
* Description: Product (item) name. Nominal.
* Quantity: The quantities of each product (item) per transaction. Numeric.
* InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
* UnitPrice: Unit price. Numeric, Product price per unit in sterling.
* CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
* Country: Country name. Nominal, the name of the country where each customer resides.

#### 2.1 Countries
Let's take a look at the countries from which orders were made.

In [54]:
customer_data['Country'].value_counts()


United Kingdom          356728
Germany                   9480
France                    8475
EIRE                      7475
Spain                     2528
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1471
Australia                 1258
Norway                    1086
Italy                      803
Channel Islands            757
Finland                    695
Cyprus                     611
Sweden                     461
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     247
Unspecified                241
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45
Lithuani

In [61]:
!pip install opencage


  Attempting uninstall: Requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

google-api-core 1.22.2 requires protobuf>=3.12.0, but you'll have protobuf 3.11.2 which is incompatible.


In [ ]:
from opencage.geocoder import OpenCageGeocode

# Set up the OpenCage Geocoder API key
api_key = '3a62e55df6254668b3df25a7a1d9cdd5'  
geocoder = OpenCageGeocode(api_key)

map = folium.Map()

# Iterate over the countries in your DataFrame column
for country in customer_data['Country']:
    results = geocoder.geocode(country)
    
    if len(results) > 0:
        result = results[0]
        lat = result['geometry']['lat']
        lon = result['geometry']['lng']
        
        marker = folium.Marker(location=[lat, lon], popup=country)
        marker.add_to(map)

# Display the map
map
